In [2]:
# importing libraries
import csv
from bs4 import BeautifulSoup

# Firefox & Chorme
from selenium import webdriver

import time

import pandas as pd 

In [15]:
#Startup the driver
driver = webdriver.Chrome('C:/webdriver/chromedriver.exe')
url = 'https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&otracker=categorytree&page=1'
driver.get(url)

In [12]:
#Get link by search term
def get_url(search_term):
    """generate a url from search term"""
    template = 'https://www.flipkart.com/{}/pr?sid=tyy%2C4io&otracker=categorytree&page=1'
    return template.format(search_term)

In [13]:
url = get_url('mobiles')
print(url)

https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&otracker=categorytree&page=1


## Extract the collection

In [16]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [22]:
results = soup.find_all('div',{'class':'_13oc-S'})

### Extract the single data

In [23]:
item = results[0]

In [33]:
# URL of a product
atag = item.find('a',{'class':'_1fQZEK'})
url = 'https://www.flipkart.com'+atag.get('href')
print(url)

https://www.flipkart.com/redmi-9i-midnight-black-64-gb/p/itm0e1018dac2627?pid=MOBFV8RYKWQ3HACE&lid=LSTMOBFV8RYKWQ3HACEV2QOWQ&marketplace=FLIPKART&store=tyy%2F4io&srno=b_1_1&otracker=browse&fm=organic&iid=157a0d42-44cd-47a0-b6c7-4724442fa11e.MOBFV8RYKWQ3HACE.SEARCH&ppt=None&ppn=None&ssid=g5kb8oj0f40000001624007423337


In [32]:
# URL of a product image
imgtag = item.find('img',{'class':'_396cs4 _3exPp9'})
imgurl = imgtag.get('src')
print(imgurl)

https://rukminim1.flixcart.com/image/312/312/kesv0y80/mobile/h/p/q/redmi-9i-mzb0814in-original-imafvehkgzghyqtp.jpeg?q=70


In [27]:
# Name of a product
name = item.find('div',{'class':'_4rR01T'}).text
name

'REDMI 9i (Midnight Black, 64 GB)'

In [28]:
# Original Price
OriginalPrice = item.find('div',{'class':'_30jeq3 _1_WHN1'}).text
OriginalPrice

'₹8,299'

In [29]:
# Saling price
SalesPrice = item.find('div',{'class':'_3I9_wc _27UcVY'}).text
SalesPrice

'₹9,999'

In [34]:
# Rating
Rating = item.find('div',{'class':'_3LWZlK'}).text
Rating

'4.3'

# Web Scraping

In [58]:
# importing libraries
import csv
from bs4 import BeautifulSoup
# Firefox & Chorme
from selenium import webdriver
import time

def get_url(search_term):
    """generate a url from search term"""
    
    #link
    template = 'https://www.flipkart.com/{}/pr?sid=tyy,4io&otracker=categorytree'
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page={}'
    
    return url


def extract_record(item):
    """Extract and return data from a single record"""
    
    # URL of a product
    atag = item.find('a',{'class':'_1fQZEK'})
    url = 'https://www.flipkart.com'+atag.get('href')
    
    # URL of a product image
    imgtag = item.find('img',{'class':'_396cs4 _3exPp9'})
    img_url = imgtag.get('src')

    # Name of a product
    try :
        product_name = item.find('div',{'class':'_4rR01T'}).text
    except AttributeError:
        product_name = 'NAN'
   
    # Saling Price
    try :
        SalesPrice = item.find('div',{'class':'_3I9_wc _27UcVY'}).text
    except AttributeError:
        SalesPrice = 'NAN'
    
    # Original Price
    try:
        OriginalPrice = item.find('div',{'class':'_30jeq3 _1_WHN1'}).text
    except AttributeError:
        OriginalPrice = 'NAN'
        

    # Product Rating
    try:
        Rating = item.find('div',{'class':'_3LWZlK'}).text
    except AttributeError:
        Rating = 'NAN'
        
        
    Productdetails = (product_name,SalesPrice,OriginalPrice,Rating,url,img_url)

    return Productdetails

def main(search_term):
    """Run main program routine"""
    # startup the webdriver
    driver = webdriver.Chrome('C:/webdriver/chromedriver.exe')
    
    # store the all data 
    productList = []
    url = get_url(search_term)
    
    # Iterate page no. 1 to 42
    for page in range(1,42):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source,'html.parser')
        results = soup.find_all('div',{'class':'_13oc-S'})
        
        for item in results:
            record = extract_record(item)
            if record:
                productList.append(record)
                # sleep for 1sec
                time.sleep(1)
                
    driver.close()
    
    # Create CSV file
    with open('FlipkartProduct.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Name','SalesPrice','OriginalPrice','Rating','ProductLink','ImageLink'])
        writer.writerows(productList)
    

In [59]:
# run main function using search term
main('mobiles')

In [3]:
import pandas as pd
df = pd.read_csv('FlipkartProduct.csv')
df.head(20)

,Name,SalesPrice,OriginalPrice,Rating,ProductLink,ImageLink
0,"REDMI 9i (Midnight Black, 64 GB)","₹9,999","₹8,299",4.3,https://www.flipkart.com/redmi-9i-midnight-bla...,https://rukminim1.flixcart.com/image/312/312/k...
1,"Infinix Hot 10 (Ocean Wave, 64 GB)","₹11,999","₹8,999",4.4,https://www.flipkart.com/infinix-hot-10-ocean-...,https://rukminim1.flixcart.com/image/312/312/k...
2,"Infinix Hot 10 (Moonlight Jade, 64 GB)","₹11,999","₹8,999",4.4,https://www.flipkart.com/infinix-hot-10-moonli...,https://rukminim1.flixcart.com/image/312/312/k...
3,"I Kall K400 (Blue, 64 GB)","₹6,499","₹6,299",3,https://www.flipkart.com/kall-k400-blue-64-gb/...,https://rukminim1.flixcart.com/image/312/312/k...
4,"Infinix Hot 10 (Amber Red, 64 GB)","₹11,999","₹8,999",4.4,https://www.flipkart.com/infinix-hot-10-amber-...,https://rukminim1.flixcart.com/image/312/312/k...
5,"Infinix Hot 10 (Obsidian Black, 64 GB)","₹11,999","₹8,999",4.4,https://www.flipkart.com/infinix-hot-10-obsidi...,https://rukminim1.flixcart.com/image/312/312/k...
6,"realme C20 (Cool Grey, 32 GB)","₹7,999","₹6,799",4.4,https://www.flipkart.com/realme-c20-cool-grey-...,https://rukminim1.flixcart.com/image/312/312/k...
7,"I Kall K 320 (Purple, 64 GB)",NAN,"₹6,799",3.2,https://www.flipkart.com/kall-k-320-purple-64-...,https://rukminim1.flixcart.com/image/312/312/k...
8,"realme C20 (Cool Blue, 32 GB)","₹7,999","₹6,799",4.4,https://www.flipkart.com/realme-c20-cool-blue-...,https://rukminim1.flixcart.com/image/312/312/k...
9,"realme C21 (Cross Black, 32 GB)","₹9,999","₹7,999",4.5,https://www.flipkart.com/realme-c21-cross-blac...,https://rukminim1.flixcart.com/image/312/312/k...


In [7]:
df['SalesPrice'] = df['SalesPrice'].str.replace(r"[₹,,]",'')

<ipython-input-7-850700444e41>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SalesPrice'] = df['SalesPrice'].str.replace(r"[₹,,]",'')


In [9]:
df['OriginalPrice'] = df['OriginalPrice'].str.replace(r"[₹,,]",'')

<ipython-input-9-98f775e9c918>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['OriginalPrice'] = df['OriginalPrice'].str.replace(r"[₹,,]",'')


In [10]:
df.head(20)

,Name,SalesPrice,OriginalPrice,Rating,ProductLink,ImageLink
0,"REDMI 9i (Midnight Black, 64 GB)",9999,8299,4.3,https://www.flipkart.com/redmi-9i-midnight-bla...,https://rukminim1.flixcart.com/image/312/312/k...
1,"Infinix Hot 10 (Ocean Wave, 64 GB)",11999,8999,4.4,https://www.flipkart.com/infinix-hot-10-ocean-...,https://rukminim1.flixcart.com/image/312/312/k...
2,"Infinix Hot 10 (Moonlight Jade, 64 GB)",11999,8999,4.4,https://www.flipkart.com/infinix-hot-10-moonli...,https://rukminim1.flixcart.com/image/312/312/k...
3,"I Kall K400 (Blue, 64 GB)",6499,6299,3,https://www.flipkart.com/kall-k400-blue-64-gb/...,https://rukminim1.flixcart.com/image/312/312/k...
4,"Infinix Hot 10 (Amber Red, 64 GB)",11999,8999,4.4,https://www.flipkart.com/infinix-hot-10-amber-...,https://rukminim1.flixcart.com/image/312/312/k...
5,"Infinix Hot 10 (Obsidian Black, 64 GB)",11999,8999,4.4,https://www.flipkart.com/infinix-hot-10-obsidi...,https://rukminim1.flixcart.com/image/312/312/k...
6,"realme C20 (Cool Grey, 32 GB)",7999,6799,4.4,https://www.flipkart.com/realme-c20-cool-grey-...,https://rukminim1.flixcart.com/image/312/312/k...
7,"I Kall K 320 (Purple, 64 GB)",NAN,6799,3.2,https://www.flipkart.com/kall-k-320-purple-64-...,https://rukminim1.flixcart.com/image/312/312/k...
8,"realme C20 (Cool Blue, 32 GB)",7999,6799,4.4,https://www.flipkart.com/realme-c20-cool-blue-...,https://rukminim1.flixcart.com/image/312/312/k...
9,"realme C21 (Cross Black, 32 GB)",9999,7999,4.5,https://www.flipkart.com/realme-c21-cross-blac...,https://rukminim1.flixcart.com/image/312/312/k...


In [11]:
df.to_csv('FlipkartData.csv',index=0)